# Transfer gene hits from one bed file to another

### This notebook reads blastn/blastp results and transfers genes from the reference bed file to the needed bed file. To get these results, follow Viera's instructions:

>1.) B. spiz. - prepare CDS (Coding DNA Sequence) fasta file using one of the commands below (Sometimes these files are attached in the NCBI reference annotation DB and can be downloaded):
>
>a.) bedtools getfasta
bedtools getfasta -fi reference.fasta -bed W23wt_June2021.bed -fo cds_sequences.fasta
>
>OR
>b.) Convert BED to GFF Format, use 'extractfeat' function (tool) from EMBOSS package extractfeat -sequence reference.fasta -feat CDS -source annotation.gff -outseq cds_sequences.fasta
>
>2.)  Compare CDS from B. subtilis and B. spiz. using NCBI blastn (e-value should be 1e-6 at least; call only the best hit)
>
>-> Instead of the whole genome reference_genome.fasta for B. subtilis,
>use also here CDS file for both species. The headers of B. subtilis CDS sequences should contain IDs of B. subtilis genes.
>
>a.) build reference for B. subtilis
>
>makeblastdb -in reference_genome_CDS.fasta -dbtype nucl -out reference_db_CDS
>
>b.) compare (query) CDS of B. spiz. to the reference genome; one best hit per B. spiz. CDS sequence
>
>blastn -query cds_sequences.fasta -db reference_db_CDS -out blast_results.txt -outfmt 6 -evalue 1e-6 -max_target_seqs 1
>
>Sequences not found in the blastn results are very specific to B.  
>spiz. Use B. subt. IDs from the results and add them to the B. spiz.  
>annotation file (bed, gtf).
>
>Do you have a conversion table of gene IDs used for B. subtilis (NCBI, Ensembl, other common)?


## Executed
```bedtools getfasta -fi W23wt.fasta -bed W23wt_June2021.bed.txt -fo W23wt_cds_sequences.fasta
bedtools getfasta -fi Bs166NCe.fasta -bed Bs166NCe_June2021.bed.txt -fo Bs166NCe_cds_sequences.fasta
bedtools getfasta -fi "/mnt/c/Users/Philipp/Downloads/GCF_000009045.1/GCF_000009045.1_ASM904v1_genomic.fna" -bed Bs166NCe_June2021.bed.txt -fo Bs166NCe_cds_sequences.fasta
bedtools getfasta -fi "/mnt/c/Users/Philipp/Downloads/GCF_006094475.1/GCF_006094475.1_ASM609447v1_genomic.fna" -bed W23wt_June2021.bed.txt -fo W23wt_cds_sequences.fasta

makeblastdb -in Bsub_GCF_000009045.1_ASM904v1_genomic.fna -dbtype nucl -out Bsub_reference_db_CDS

blastn -query Bsub_GCF_000009045.1_ASM904v1_genomic.fna -db Bsub_reference_db_CDS -out blast_results.txt -outfmt 6 -evalue 1e-6 -max_target_seqs 1



# to get blastp database
makeblastdb -in Bsub_GCF_000009045.1_ASM904v1_genomic.fna -dbtype prot -out Bsub_protein_reference_db_CDS
```

# Actual code

## Ansatz

- iterate through the bed file you want to annotate
- for each line get the start and stop position
- check if the interval is found in blast results (+/- 3 nucleotides)
    - 3rd column = pident = should be > 95?%
    - 7th/8th column = start/stop query, position in CDS = useless!!!
    - 9th/10th column = start/stop subject, position in CDS = useless!!!
    - instead: use qseqid (refering to bed file; iterated line) and sseqid (refering to reference bed file)
        - example: chr1:1938-3074	chr2:1939-3075
        - use RegEx magic to extract 1938, 3074 and 1939, 3075
- if good hit in blast results
    - calculate position on reference genome bed file
        - `offset = start subject - start query`
    - lookup locus in reference bed file
    - copy gene identifier to improved bed file

In [4]:
import pandas as pd

# Load the bed file and blast results
#incomplete_bed_file = pd.read_csv("//wsl.localhost/Ubuntu/home/philipp/dicts/W23wt_June2021.bed.txt", sep='\t', header=None, names=['seqName', 'start', 'end', 'direction', 'type1', 'type2', 'name', 'locus_tag', 'product'], comment='#')
incomplete_bed_file = pd.read_csv("//wsl.localhost/Ubuntu/home/philipp/dicts/Features from W23_NCBI_comK_inducible_delta_comK.bed", sep='\t', header=None, names=['seqName', 'start', 'end', 'direction', 'type1', 'type2', 'name', 'locus_tag', 'product'], comment='#')

reference_bed_file = pd.read_csv("//wsl.localhost/Ubuntu/home/philipp/dicts/Bs166NCe_June2021.bed.txt", sep='\t', header=None, names=['seqName', 'start', 'end', 'direction', 'type1', 'type2', 'name', 'locus_tag', 'product'], comment='#')

blast_filename = "//wsl.localhost/Ubuntu/home/philipp/dicts/blast_results_MelihFasta.txt"
#blast_filename = "//wsl.localhost/Ubuntu/home/philipp/dicts/blast_chr_results.txt"
blast_results = pd.read_csv(blast_filename, sep='\t', header=None, names=['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore'], comment='#')

# Filter blast results for high identity matches
blast_results = blast_results[blast_results['pident'] > 80]

# create new columns with the real loci
blast_results["real_qstart"] = blast_results["qseqid"].str.extract(r'[\w.-]+:(\d+)-\d+').astype(int)
blast_results["real_qend"] = blast_results["qseqid"].str.extract(r'[\w.-]+:\d+-(\d+)').astype(int)
blast_results["real_sstart"] = blast_results["sseqid"].str.extract(r'[\w.-]+:(\d+)-\d+').astype(int)
blast_results["real_send"] = blast_results["sseqid"].str.extract(r'[\w.-]+:\d+-(\d+)').astype(int)

#incomplete_bed_file = incomplete_bed_file[0:8]

# Create a new DataFrame for the improved bed file
##improved_bed = pd.DataFrame(columns=['seqName', 'start', 'end', 'direction', 'type1', 'type2', 'name', 'locus_tag', 'product'])
improved_bed = pd.DataFrame(columns=incomplete_bed_file.columns) # copy columns from bed_file
improved_bed = improved_bed.astype(incomplete_bed_file.dtypes.to_dict()) # copy dtypes from bed_file

# Iterate through the bed file
for index, row in incomplete_bed_file.iterrows():
    start = row['start']
    end = row['end']
    
    # Check if the interval is found in blast results (+/- 3 nucleotides)
    match = blast_results[(blast_results['real_qstart'] <= start + 3) & (blast_results['real_qend'] >= end - 3)]
    #print("match: ", start , end,  "\n", match[["real_qstart", "real_qend"]])

    
    if not match.empty:    
        #if len(match) > 1:
        #    print(f"Multiple matches found for {row['name']} between {start} and {end}:\n{match}")
        
        # Get the shortest match
        blast_row = match.loc[match['length'].idxmin()]
        
        # Calculate position on reference genome
        offset = blast_row['real_sstart'] - blast_row['real_qstart']
        ref_start = start + offset
        ref_end = end + offset
        
        # Lookup locus in reference bed file (+/- 3 nucleotides)
        ref_locus = reference_bed_file[(reference_bed_file['start'] >= ref_start - 3) & (reference_bed_file['end'] <= ref_end + 3)]
        
        #if ref_locus.empty:
        #    print(print(f"Found NO locus for {row['name']}"))
        
        if not ref_locus.empty:
            #print(f"Found locus for {row['name']}")
            #print(row['seqName'])
            # Copy gene identifier to improved bed file
            improved_bed = pd.concat([improved_bed, pd.DataFrame({
                'seqName': ref_locus.iloc[0]['seqName'],
                'start': row['start'],
                'end': row['end'],
                'direction': ref_locus.iloc[0]['direction'],
                'type1': ref_locus.iloc[0]['type1'],
                'type2': ref_locus.iloc[0]['type2'],
                'name': ref_locus.iloc[0]['name'],
                'locus_tag': ref_locus.iloc[0]['locus_tag'],
                'product': ref_locus.iloc[0]['product'],
                'old_name' : row['name'],
                'old_locus_tag' : row['locus_tag'],
                'old_product' : row['product']
            }, index=[0])], ignore_index=True)

# Modify the first column name to prepend '#'
# This is to get the '#' as first character in the header
new_header = ['#' + improved_bed.columns[0]] + list(improved_bed.columns[1:])

# Save the improved bed file
improved_bed.to_csv('improved_bed_file_MelihFASTA.bed', sep='\t', header=new_header, index=False)

In [2]:
improved_bed.head()

,seqName,start,end,direction,type1,type2,name,locus_tag,product,old_name,old_locus_tag,old_product
0,NC_000964.3,410,1750,+,CDS,gene,dnaA,BSU_00010,chromosomal replication initiator informationa...,dnaA,EO946_RS00002,NaN
1,NC_000964.3,410,1750,+,CDS,gene,dnaA,BSU_00010,chromosomal replication initiator informationa...,dnaA,EO946_RS00003,NaN
2,NC_000964.3,1938,3074,+,CDS,gene,dnaN,BSU_00020,DNA polymerase III (beta subunit),dnaN,EO946_RS00004,NaN
3,NC_000964.3,1938,3074,+,CDS,gene,dnaN,BSU_00020,DNA polymerase III (beta subunit),dnaN,EO946_RS00005,NaN
4,NC_000964.3,3206,3421,+,CDS,gene,rlbA,BSU_00030,RNA binding protein involved in ribosome matur...,rlbA,EO946_RS00006,NaN


In [ ]:
# Tool to convert Snapgene's features list to mock bed file:
if False:
    
    import pandas as pd

    def convert_to_bed(input_file, output_file, reference_sequence, header_string=None):
        '''
        Convert a txt file with gene coordinates to a BED file.
        The txt file should have two columns: gene name and coordinates.
        The coordinates should be in the format "start..stop".
        The BED file will have the following columns:
        - reference
        - start
        - stop
        - strand
        - feature_type

        Parameters:
        input_file (str): Path to the input txt file.
        output_file (str): Path to the output BED file.
        reference_sequence (str): Name of the reference sequence.
        header_string (str): Optional header string to be added to the BED file.
        
        Returns:
        None
        '''
        
        # Read the txt file
        data = pd.read_csv(input_file, sep=r'\t', header=None, names=['gene', 'coordinates'], engine="python", skipinitialspace=True)

        # Split the coordinates into start and stop
        data[['start', 'stop']] = data['coordinates'].str.replace("..","+").str.replace(".","").str.split('+', expand=True).astype(int)

        # Prepare the bed format
        bed_data = pd.DataFrame({
            'reference': reference_sequence,
            'start': data['start'],
            'stop': data['stop'],
            'strand': '+', # dumbly just put + for all
            'feature_type': 'CDS', # dumbly just put CDS for all
            'feature': 'gene', # dumbly just put gene for all
            'gene_name': data['gene'].str.rstrip(), # remove trailing whitespaces
            'gene_id': ['EO946_RS' + str(i).zfill(5) for i in range(1, len(data) + 1)],
            'description': "" # ['description for ' + gene for gene in data['gene']]
        })

        # Convert DataFrame to string with tab-separated values
        bed_content = bed_data.to_csv(sep='\t', header=False, index=False, lineterminator="\r\n")

        # Combine header and BED content
        if header_string is None:
            full_content = bed_content
        else:
            full_content = header_string + "\r\n" + bed_content

        # Write to a new BED file
        with open(output_file, 'w', newline="\n") as file:
            file.write(full_content)


    # Example usage
    header_string = "# this bed file was created by hand from Snapgene's feature export from W23_NCBI_comK_inducible_delta_comK, created by Melih in November 2024\r\n\
# Bacillus spizizenii ATCC 6633 = JCM 2499 strain ATCC 6633 chromosome, complete genome. NZ_CP034943\r\n\
# seqName	start	ende	direction	type1	type2	Name	locus_tag	product"
    
    convert_to_bed("U:/home/philipp/dicts/Features from W23_NCBI_comK_inducible_delta_comK.txt", "U:/home/philipp/dicts/Features from W23_NCBI_comK_inducible_delta_comK.bed", 'NZ_CP034943.1', header_string)

    